In [31]:
import requests, json
from dataclasses import dataclass, asdict

coins = { "BTC", "BCH", "DOGE", "LTC", "KAS", "RVN"}
api_token = "8a0b92957b57d4eea9cdf73c4d7436c09c82a38865b84e5b2304b7b5a0202b92"
coins_endpoint = f"https://whattomine.com/api/v1/coins?api_token={api_token}"
json = requests.get(coins_endpoint).json()

@dataclass
class WhatToMineData:
    tag: str = ""
    block_time: float = 0
    block_reward_btc: float = 0
    difficulty: float = 0
    nethash: float = 0
    daily_emissions_btc: float = 0
    algorithm: str = ""

def toWhatToMineData(json):
    data = WhatToMineData()
    data.tag = json['tag']
    data.block_time = float(json['block_time'])
    exchange_rate_btc = 1 if data.tag == "BTC" else json['exchanges'][0]['price']
    data.block_reward_btc = json['block_reward'] * exchange_rate_btc
    data.difficulty = json['difficulty']
    data.nethash = json['nethash']
    data.daily_emissions_btc = 86400 / data.block_time * data.block_reward_btc
    data.algorithm = json['algorithm']
    return data

coins_data = [ toWhatToMineData(coin) for coin in json if coin['tag'] in coins ]
print("Global coins data:\n")
for coin_data in coins_data:
    print(asdict(coin_data))


Global coins data:

{'tag': 'BCH', 'block_time': 631.0, 'block_reward_btc': 0.010705520356198178, 'difficulty': 372150347203.83, 'nethash': 2533080143320911251, 'daily_emissions_btc': 1.4658588887092276, 'algorithm': 'SHA-256'}
{'tag': 'BTC', 'block_time': 651.0, 'block_reward_btc': 3.1561542979999997, 'difficulty': 108105433845147.0, 'nethash': 713224735614128878648, 'daily_emissions_btc': 418.8813077529954, 'algorithm': 'SHA-256'}
{'tag': 'DOGE', 'block_time': 63.0, 'block_reward_btc': 0.027346264381304417, 'difficulty': 34701022.5407986, 'nethash': 2365710427785536, 'daily_emissions_btc': 37.50344829436034, 'algorithm': 'Scrypt'}
{'tag': 'KAS', 'block_time': 1.0006, 'block_reward_btc': 6.50779807444094e-05, 'difficulty': 314858124.1855535, 'nethash': 1351494449587106735, 'daily_emissions_btc': 5.619365916766912, 'algorithm': 'kHeavyHash'}
{'tag': 'LTC', 'block_time': 150.0, 'block_reward_btc': 0.0067625, 'difficulty': 74389147.7639597, 'nethash': 2129993045490122, 'daily_emissions_b

In [32]:
import requests, json
import pandas as pd

algorithms = { "SCRYPT", "SHA256ASICBOOST", "KHEAVYHASH", "KAWPOW"}

multialgo_response = requests.get('https://api2.nicehash.com/main/api/v2/public/simplemultialgo/info')
multialgo_data = multialgo_response.json()
algo_data = { element['algorithm']: {'paying': element['paying'], 'speed': element['speed']} for element in multialgo_data['miningAlgorithms'] if element['algorithm'] in algorithms }

algorithms_response = requests.get('https://api2.nicehash.com/main/api/v2/mining/algorithms')
algorithms_response.raise_for_status()
algorithms_data = algorithms_response.json()

index = [ element['algorithm'] for element in algorithms_data['miningAlgorithms'] if element['algorithm'] in algorithms ]
columns = ['marketFactor', 'displayMarketFactor', 'priceFactor', 'displayPriceFactor', 'miningFactor', 'displayMiningFactor', 'priceDownStep', 'paying', 'speed']
data = [ [ float(element['marketFactor']), element['displayMarketFactor'], float(element['priceFactor']), element['displayPriceFactor'], float(element['miningFactor']), element['displayMiningFactor'], float(element['priceDownStep']), algo_data[element['algorithm']]['paying'], algo_data[element['algorithm']]['speed'] ] for element in algorithms_data['miningAlgorithms'] if element['algorithm'] in algorithms ]

raw_data_df = pd.DataFrame(data, index=index, columns=columns)

calc_df = pd.DataFrame(index=index)
sats2btc = 1e8

calc_df['hashrate'] = raw_data_df['speed'] / raw_data_df['marketFactor'] * raw_data_df['miningFactor']
calc_df['hashrate unit'] = raw_data_df['displayMarketFactor'] + '/s'

calc_df['paying'] = raw_data_df['paying'] * raw_data_df['priceFactor'] / sats2btc
calc_df['paying unit'] = 'BTC/' + raw_data_df['displayPriceFactor'] + '/day'

print("Nicehash data:\n")
json.loads(calc_df.to_json(orient='index'))


Nicehash data:



{'SCRYPT': {'hashrate': 98.9801217343,
  'hashrate unit': 'TH/s',
  'paying': 0.018830797,
  'paying unit': 'BTC/TH/day'},
 'SHA256ASICBOOST': {'hashrate': 6.6484500153,
  'hashrate unit': 'EH/s',
  'paying': 0.5694867655,
  'paying unit': 'BTC/EH/day'},
 'KAWPOW': {'hashrate': 0.8486558756,
  'hashrate unit': 'TH/s',
  'paying': 0.0848944021,
  'paying unit': 'BTC/TH/day'},
 'KHEAVYHASH': {'hashrate': 55.5161746058,
  'hashrate unit': 'PH/s',
  'paying': 0.0041005311,
  'paying unit': 'BTC/PH/day'}}